## 0.3 Elaborazione PDF con Dolphin OCR

Utilizziamo Dolphin per convertire i tuoi file PDF in testo. Assicurati di avere i tuoi file PDF in una cartella su Google Drive.

In [ ]:
!git clone https://github.com/ByteDance/Dolphin.git
!cd Dolphin

fatal: destination path 'Dolphin' already exists and is not an empty directory.


In [ ]:
print("Tentativo di installare pymupdf separatamente...")
!pip install pymupdf==1.26 --quiet
print("Installazione pymupdf completata (o errore mostrato sopra).")

Tentativo di installare pymupdf separatamente...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 115.4 MB/s eta 0:00:00
Installazione pymupdf completata (o errore mostrato sopra).


In [ ]:
# 0.2 Installazione delle Dipendenze di Dolphin
print("Installazione delle dipendenze di Dolphin...")

# Naviga nella directory di Dolphin
%cd Dolphin

# Installa le dipendenze richieste
!pip install -r requirements.txt --quiet

# Torna alla directory principale
%cd ..

print("Installazione delle dipendenze di Dolphin completata.")

Installazione delle dipendenze di Dolphin...
/content/Dolphin
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 127.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
/content
Installazione delle dipendenze di Dolphin completata.


In [ ]:
# Download the model from Hugging Face Hub
# Install git-lfs
!git lfs install
# Clone the model repository
!git clone https://huggingface.co/ByteDance/Dolphin-1.5 ./hf_model
# Or use the Hugging Face CLI
!pip install huggingface_hub
!huggingface-cli download ByteDance/Dolphin-1.5 --local-dir ./hf_model

Git LFS initialized.
Cloning into './hf_model'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 16 (delta 2), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (16/16), 986.43 KiB | 1.81 MiB/s, done.
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 9 files:   0% 0/9 [00:00<?, ?it/s]Downloading 'README.md' to 'hf_model/.cache/huggingface/download/Xn7B-BWUGOee2Y6hCZtEhtFu4BE=.0a239101345021ac3a9d09de84a2e0dbf9b86831.incomplete'

preprocessor_config.json: 100% 477/477 [00:00<00:00, 3.83MB/s]
Download complete. Moving file to hf_model/preprocessor_config.json

README.md: 3.51kB [00:00, 13.7MB/s]
Download complete. Moving file to hf_model/README.md

generation_config.json: 100% 160/160 [00:00<00:00, 1.55MB/s]
Download complete. Moving file to hf_model/generation_config.json

config.json: 4.85kB [00:00, 20.0MB/s]
Download complete. M

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 0.3 Elaborazione PDF con Dolphin OCR (Utilizzando directory locale /content/)
import os
import glob
import shutil
# tempfile non più necessario per copiare se il file è già in /content/

# !!! MODIFICA QUESTI PERCORSI SE NECESSARIO !!!
# Percorso del file PDF di input singolo nella directory locale di Colab
# Assicurati di caricare il file qui!
PDF_INPUT_FILE = "/content/drive/MyDrive/LLM/ALFA.pdf" # Percorso del tuo file PDF in /content/

# Percorso della cartella dove verranno salvati i file di testo generati dall'OCR nella directory locale di Colab
# Questa sarà la cartella che userai nella Fase 1 per caricare i documenti
OCR_OUTPUT_PATH = "/content/drive/MyDrive/LLM/Testo_OCR" # Cartella di output in /content/

# Assicurati che la cartella di output esista (or creala)
if not os.path.exists(OCR_OUTPUT_PATH):
    os.makedirs(OCR_OUTPUT_PATH)
    print(f"Creata cartella di output OCR locale: {OCR_OUTPUT_PATH}")
else:
    print(f"Cartella di output OCR locale esistente: {OCR_OUTPUT_PATH}")

# Percorso dello script di Dolphin per l'elaborazione dei documenti (usiamo demo_page.py per estrazione testo)
DOLPHIN_SCRIPT_PATH = "/content/Dolphin/demo_page.py" # Usiamo demo_page.py

if not os.path.exists(DOLPHIN_SCRIPT_PATH):
    print(f"ERRORE: Script Dolphin non trovato al percorso specificato: {DOLPHIN_SCRIPT_PATH}")
    print("Verifica il percorso dello script 'demo_page.py' (o simile) all'interno della cartella Dolphin clonata.")
else:
    print(f"Script Dolphin trovato: {DOLPHIN_SCRIPT_PATH}")

    # Verifica se il file PDF di input esiste nella directory locale
    if not os.path.exists(PDF_INPUT_FILE):
        print(f"ERRORE: File PDF non trovato al percorso specificato: {PDF_INPUT_FILE}")
        print(f"Assicurati di aver caricato il file '{os.path.basename(PDF_INPUT_FILE)}' direttamente nella directory /content/.")
        # Non c'è bisogno del controllo IsADirectoryError per /content/ di solito
    else:
        print(f"Trovato file PDF da elaborare: {PDF_INPUT_FILE}")
        print("Inizio elaborazione OCR con Dolphin (usando demo_page.py)...")

        # Percorso del modello scaricato localmente
        # Verifica dove il modello è finito effettivamente dopo il clone
        MODEL_LOCAL_PATH = "/content/hf_model" # VERIFICA CHE QUESTO PERCORSO SIA CORRETTO!

        if not os.path.exists(MODEL_LOCAL_PATH):
             print(f"ERRORE: Modello Dolphin non trovato al percorso specificato: {MODEL_LOCAL_PATH}")
             print("Assicurati che la cella di download del modello sia stata eseguita correttamente e che i file siano in questa cartella.")
        else:
            print(f"Modello Dolphin trovato in: {MODEL_LOCAL_PATH}")

            # --- Logica per elaborare il singolo file PDF direttamente ---
            # Usiamo il comando basato sull'esempio del README per demo_page.py con input file singolo.

            print("\nEsecuzione di demo_page.py...")
            # Comando: !python "{DOLPHIN_SCRIPT_PATH}" --model_path "{MODEL_LOCAL_PATH}" --save_dir "{OCR_OUTPUT_PATH}" --input_path "{PDF_INPUT_FILE}"

            # ESECUZIONE DEL COMANDO DOLPHIN
            # Passiamo direttamente il percorso del file PDF a --input_path
            # Passiamo la cartella di output a --save_dir
            # Passiamo il percorso locale del modello a --model_path
            print(f"Comando: !python \"{DOLPHIN_SCRIPT_PATH}\" --model_path \"{MODEL_LOCAL_PATH}\" --save_dir \"{OCR_OUTPUT_PATH}\" --input_path \"{PDF_INPUT_FILE}\"")
            !python "{DOLPHIN_SCRIPT_PATH}" --model_path "{MODEL_LOCAL_PATH}" --save_dir "{OCR_OUTPUT_PATH}" --input_path "{PDF_INPUT_FILE}"

            # Dopo l'esecuzione, dobbiamo verificare cosa è stato salvato nella cartella OCR_OUTPUT_PATH
            # demo_page.py con input PDF singolo dovrebbe salvare un file JSON o Markdown (verifica nel codice/doc)
            # Potrebbe essere necessario un passaggio aggiuntivo per convertirlo in file .txt per il RAG.
            print(f"\nElaborazione (tentata) completata per {os.path.basename(PDF_INPUT_FILE)}. Controlla la cartella {OCR_OUTPUT_PATH} per i file generati da Dolphin.")


        print("\nFase di elaborazione OCR con Dolphin completata.")
        print(f"Controlla la cartella {OCR_OUTPUT_PATH} per i risultati.")
        print("Potrebbe essere necessario un passaggio aggiuntivo per convertire l'output di Dolphin (es. JSON/Markdown) in file .txt se non li crea direttamente.")

        # Ora puoi puntare la variabile DATA_PATH nella cella successiva (Fase 1) a OCR_OUTPUT_PATH

Cartella di output OCR locale esistente: /content/drive/MyDrive/LLM/Testo_OCR
Script Dolphin trovato: /content/Dolphin/demo_page.py
Trovato file PDF da elaborare: /content/drive/MyDrive/LLM/ALFA.pdf
Inizio elaborazione OCR con Dolphin (usando demo_page.py)...
Modello Dolphin trovato in: /content/hf_model

Esecuzione di demo_page.py...
Comando: !python "/content/Dolphin/demo_page.py" --model_path "/content/hf_model" --save_dir "/content/drive/MyDrive/LLM/Testo_OCR" --input_path "/content/drive/MyDrive/LLM/ALFA.pdf"
2025-11-07 14:05:54.286414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762524354.319064    2668 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762524354.329631    2668 cuda_blas.cc:1407] Unable to register cuBLAS factory: Att


-----

# 🗺️ Roadmap: Sistema RAG Automotive (da Colab a GitHub)

Questa è la versione per Google Colab della roadmap. Esegui le celle di codice in ordine.

# Fase 1: Fondamenta e Dati

L'obiettivo di questa fase è caricare i tuoi dati e prepararli per l'indicizzazione.

## 1\. Configurazione Ambiente Colab

Per prima cosa, assicurati di avere una GPU gratuita.

1.  Vai su **Runtime** (o "Ambiente di esecuzione") nel menu in alto.
2.  Seleziona **Modifica tipo di ambiente di esecuzione** (o "Change runtime type").
3.  Nel menu a discesa "Acceleratore hardware", scegli **T4 GPU**.
4.  Fai clic su "Salva".

Ora, installiamo tutte le librerie necessarie.

In [ ]:
# 1. Installazione delle librerie
# Usiamo 'quiet' per ridurre l'output di installazione
print("Installazione delle librerie necessarie...")

# Install a compatible version of requests first
!pip install requests==2.32.4 --quiet

!pip install langchain langchain-community langchain-huggingface --quiet
!pip install sentence-transformers --quiet
!pip install chromadb --quiet
!pip install transformers accelerate bitsandbytes --quiet # Per caricare l'LLM

print("Installazione completata.")

Installazione delle librerie necessarie...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.31 requires requests<3.0.0,>=2.32.5, but you have requests 2.32.4 which is incompatible.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-proto==1.37.0, but you have opentelemetry-proto 1.38.0 which is incompatible.
opentel

## 2\. Preparazione e Caricamento Dati

Questa sezione presuppone che tu abbia già eseguito il tuo script `esegui_ocr.py` in locale e abbia una cartella chiamata `documenti_txt` sul tuo Google Drive.

In [ ]:
# 2. Monta Google Drive
# Eseguendo questa cella, ti verrà chiesto di autorizzare Colab
# ad accedere al tuo Google Drive.
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive montato.")

Mounted at /content/drive
Google Drive montato.


In [ ]:
# 3. Caricamento dei Documenti (Document Loading)
from langchain.document_loaders import DirectoryLoader, TextLoader
import os

# !!! MODIFICA QUESTO PERCORSO SE NECESSARIO !!!
# Ora puntiamo alla sottocartella 'markdown' dove Dolphin ha salvato l'output testuale
# Assicurati che questo sia il percorso corretto.
DATA_PATH = "/content/drive/MyDrive/LLM/Testo_OCR/markdown" # Usa il percorso della sottocartella markdown!

if not os.path.exists(DATA_PATH):
    print(f"ATTENZIONE: Il percorso '{DATA_PATH}' non esiste.")
    print("Per favore, crea la cartella o assicurati che la cella di output OCR abbia creato questa sottocartella.")
else:
    print(f"Caricamento documenti da: {DATA_PATH}")
    loader = DirectoryLoader(
        DATA_PATH,
        glob="**/*.md", # Cerca tutti i file .md
        loader_cls=TextLoader,
        # loader_kwargs={'encoding': 'utf-8'}, # Questo argomento non era supportato qui
        show_progress=True,
        use_multithreading=True # Accelera il caricamento
    )
    documents = loader.load()
    print(f"Caricati {len(documents)} documenti.")

    # Controllo rapido
    if documents:
        print("\n--- Esempio di documento caricato ---")
        print(f"Fonte: {documents[0].metadata.get('source', 'N/A')}")
        print(f"Contenuto (primi 150 caratteri): {documents[0].page_content[:150]}...")
        print("-----------------------------------")
    else:
        print("Nessun documento trovato nella cartella specificata.")

Caricamento documenti da: /content/drive/MyDrive/LLM/Testo_OCR/markdown


100%|██████████| 1/1 [00:00<00:00, 207.56it/s]

Caricati 1 documenti.

--- Esempio di documento caricato ---
Fonte: /content/drive/MyDrive/LLM/Testo_OCR/markdown/ALFA.md
Contenuto (primi 150 caratteri): ![Figure](figures/ALFA_page_001_figure_000.png)

![Figure](figures/ALFA_page_001_figure_001.png)

ALFA ROMEC DOO BERLINA

![Figure](figures/ALFA_page_...
-----------------------------------


In [ ]:
# Assicurati che langchain-community sia installato per i Document Loaders
!pip install langchain-community --quiet
print("Installato langchain-community.")

Installato langchain-community.


In [ ]:
# 4. Divisione dei Testi (Text Chunking)
# Dividiamo i documenti in pezzi più piccoli (chunk)
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,       # Dimensione chunk (come da tuo HTML, 500-1000)
    chunk_overlap=150,      # Overlap per mantenere il contesto
    separators=["\n\n", "\n", ". ", " ", ""], # Come dividere il testo
    length_function=len
)

# Verifica se i documenti sono stati caricati prima di splittare
if 'documents' in locals() and documents:
    chunks = text_splitter.split_documents(documents)
    print(f"Documenti divisi in {len(chunks)} chunk.")

    # Controllo rapido
    print("\n--- Esempio di chunk ---")
    print(chunks[0].page_content)
    print("------------------------")
else:
    print("Nessun documento caricato. Salto il chunking.")

Documenti divisi in 131 chunk.

--- Esempio di chunk ---
![Figure](figures/ALFA_page_001_figure_000.png)

![Figure](figures/ALFA_page_001_figure_001.png)

ALFA ROMEC DOO BERLINA

![Figure](figures/ALFA_page_001_figure_003.png)

USO E MANUTENZIONI

---

## ATTENZIONE

all'ossido di carbonio!

Non tenete mai in moto il motore in un locale chiuso.

I gas di scarico contengono ossido di carbonio, gas fortemente tossico ed inoltre molto pericoloso perché, essendo incolore e insapore, è difficile avvertirne la presenza.

Consigliamo prendere nota della sigla stampigliata sull'impugnatura delle chiavi.

Interruttore di accensione e blocca-sterzo.

![Figure](figures/ALFA_page_002_figure_004.png)

Porte lato guida e passeggero; cassettino del cruscotto; bagagliera.

![Figure](figures/ALFA_page_002_figure_006.png)
------------------------


# Fase 2: Costruzione della Pipeline RAG

Ora indicizziamo i chunk e costruiamo la catena di Q\&A.

In [ ]:

!pip install langchain langchain-community langchain-huggingface --quiet
print("Reinstallazione completata.")

Reinstallazione completata.


In [ ]:
# 5. Modello di Embedding (Gratuito)
# Convertiamo i chunk di testo in vettori (numeri)
from langchain_huggingface import HuggingFaceEmbeddings
import torch

# Controlla se la GPU è disponibile e imposta il dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Utilizzo del dispositivo: {device}")

model_name = "intfloat/multilingual-e5-large"
model_kwargs = {'device': device} # Usa la GPU T4 di Colab
encode_kwargs = {'normalize_embeddings': True} # Importante per E5

print("Caricamento modello di embedding...")
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

print("Modello di embedding caricato.")

Utilizzo del dispositivo: cuda
Caricamento modello di embedding...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Modello di embedding caricato.


In [ ]:
!pip install -U bitsandbytes

In [ ]:
# 7. LLM (Generazione Risposta - Gratuito)
# Usiamo un modello quantizzato per farlo stare nella GPU di Colab
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig # Importa BitsAndBytesConfig
import torch

# Usiamo un modello 7B quantizzato (GGUF) che sta bene nella GPU T4
# Nota: L'uso di GGUF richiede 'ctransformers', ma 'bitsandbytes' (già installato)
# ci permette di caricare modelli quantizzati standard.
# Usiamo Mistral-7B-Instruct-v0.2 quantizzato a 4-bit.

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

print(f"Caricamento LLM: {model_id} (quantizzato a 4-bit)...")

# Configurazione per la quantizzazione a 4-bit (usando BitsAndBytesConfig)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Carichiamo il modello con la configurazione di quantizzazione
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # Usa quantization_config invece di load_in_4bit
    device_map="auto" # Mette il modello sulla GPU
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Creiamo la pipeline di text-generation
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024, # Aumentato per risposte più dettagliate
    temperature=0.1,
    repetition_penalty=1.1
)

# Integriamo la pipeline in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print("LLM caricato e pronto.")

Caricamento LLM: mistralai/Mistral-7B-Instruct-v0.2 (quantizzato a 4-bit)...


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0


LLM caricato e pronto.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 8. Creazione della RAG Chain (LangChain)
# Assembliamo tutti i pezzi
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Il tuo prompt template (come da index.html)
template = """Usa il seguente contesto per rispondere alla domanda
sull'auto d'epoca. Fornisci una risposta dettagliata e tecnica basata
esclusivamente sul contesto, se capisci che la domanda non è chiara o non trovi nulla che si avvicina alla domanda dichiara assolutamente che non riesci a trovare l'informazione.
IMPORTANTE! Se non sai la risposta o non è nel contesto,
dillo chiaramente SENZA CONSIDERARLO UN FALLIMENTO. ASSOLUTAMENTE NON inventare informazioni!.
ATTIENITI SOLO ED ESCLUSIVAMENTE ALLE INFORMAZIONI E ALLE FOTO CHE RIESCI A TROVARE E CHE SI ROCLLEGANO STRETTAMENTE ALLA QUERY.

Contesto:
{context}

Domanda: {question}

Risposta dettagliata in italiano:"""

PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

# Creiamo il retriever (che "cerca" nel vector store)
if vectordb:
    retriever = vectordb.as_retriever(
        search_type="similarity", # o "mmr"
        search_kwargs={"k": 5}    # Recupera i top 5 chunk
    )

    # La catena RAG finale
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff", # "stuff" inserisce tutti i chunk nel prompt
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True # Vogliamo vedere da dove proviene la risposta
    )

    print("Catena RAG creata e pronta per le domande.")
else:
    print("ERRORE: Il Vector DB non è stato caricato. Impossibile creare la catena RAG.")

Catena RAG creata e pronta per le domande.


# Fase 3: Test e Valutazione

Ora proviamo il sistema\!

In [ ]:
# 9. Test Manuale
# Prova a fare una domanda sui tuoi documenti!
###############################################
query = "a che pressione devo mettere gli pneumatici anteriori e posteriori?"
###############################################

if 'qa_chain' in locals():
    print(f"Esecuzione query: {query}")
    print("Ricerca in corso (potrebbe richiedere un po' di tempo)...")

    # Usiamo il metodo .invoke()
    result = qa_chain.invoke({"query": query})

    print("\n--- RISPOSTA ---")
    # Il modello Mistral-Instruct a volte ripete la domanda, puliamo l'output.
    # La risposta effettiva di solito inizia dopo "Risposta dettagliata in italiano:"
    risposta_pulita = result['result'].split("Risposta dettagliata in italiano:")[-1].strip()
    print(risposta_pulita)

    print("\n--- FONTI UTILIZZATE ---")
    if result['source_documents']:
        for i, doc in enumerate(result['source_documents']):
            print(f"\nFonte {i+1}: {doc.metadata.get('source', 'N/A')}")
            print(f"Contenuto: ...{doc.page_content[:250]}...")
    else:
        print("Nessun documento di origine trovato.")

else:
    print("La catena RAG (qa_chain) non è stata creata. Controlla gli errori nelle celle precedenti.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Esecuzione query: a che pressione devo mettere gli pneumatici anteriori e posteriori?
Ricerca in corso (potrebbe richiedere un po' di tempo)...

--- RISPOSTA ---
Per determinare la pressione corretta di gonfiamento per gli pneumatici anteriori e posteriori, è necessario seguire i passaggi descritti nel testo del manuale. In particolare, si deve calzare i tubetti flessibili sui viti di spurgo di una ruota anteriore e di una ruota posteriore, premere il pedale del freno e tenere premuto fino a quando non uscirà dal tubetto un liquido privo di bolle d'aria. Quindi, mantenendo premuto il pedale, bloccare le viti e togliere i tubetti. Si deve tenere presente i valori degli spazi di arresto indicati nel diagramma. Il manuale fornisce anche una tabella con i valori di pressione consigliati per diversi tipi di pneumatici. Ad esempio, per il pneumatico Firestone Cavallino Sport 200, la pressione consigliata è di 1,8 kg/cm².

In conclusione, la pressione corretta di gonfiamento per gli pneumatic

# Fase 4: Refactoring e Deploy (Note)

Questa è la fine del lavoro su Colab.

Come descritto nella roadmap, il prossimo passo (da fare **in locale** sulla tua macchina, non qui) è trasformare questo notebook in un progetto Python strutturato per GitHub:

1.  **Modularizzazione:** Dividi la logica in file `.py` separati (es. `config.py`, `pipeline_rag.py`, `ingest.py`).
2.  **Interfaccia:** Crea un'interfaccia utente semplice con Streamlit o Gradio (es. `app.py`).
3.  **Requisiti:** Crea un file `requirements.txt` con tutte le librerie usate.
4.  **README:** Scrivi un `README.md` che spiega come installare ed eseguire il progetto.

Questo notebook Colab ti ha permesso di costruire e testare l'intera pipeline RAG gratuitamente. Ora hai tutti i componenti funzionanti pronti per essere impacchettati per GitHub.

# Task
Explain how to create a repository and push changes to it.

## Set up git

### Subtask:
Ensure Git is installed and configured on your local machine.


## Create a local repository

### Subtask:
Initialize a Git repository in your project directory.


## Add and commit files

### Subtask:
Stage your project files and commit them to the local repository.


## Create a remote repository

### Subtask:
Create a new repository on a hosting service like GitHub, GitLab, or Bitbucket.


## Connect local to remote

### Subtask:
Link your local repository to the remote repository.


## Push changes

### Subtask:
Push your committed changes from the local repository to the remote repository.


## Summary:

### Data Analysis Key Findings

*   The process of creating a repository and pushing changes involves several steps: setting up Git locally, creating a local repository, adding and committing files, creating a remote repository, connecting the local and remote repositories, and finally pushing the changes.
*   Due to the limitations of the execution environment, none of the Git operations (checking installation, initializing a local repository, adding/committing files, connecting local to remote, pushing changes) could be performed directly.
*   Creating a remote repository on a hosting service also required manual steps outside of the execution environment.

### Insights or Next Steps

*   To complete the overall task, the user needs to follow the provided instructions in their local terminal for each step.
*   Future iterations of this task could benefit from an environment that allows direct interaction with the user's local file system and Git installation or provides a simulated Git environment for demonstration purposes.
